In [1]:
!pip install requests beautifulsoup4 openpyxl


     -------------------------------------- 62.6/62.6 kB 480.1 kB/s eta 0:00:00
  Using cached openpyxl-3.1.2-py2.py3-none-any.whl (249 kB)
     ---------------------------------------- 96.7/96.7 kB 1.4 MB/s eta 0:00:00
     -------------------------------------- 123.2/123.2 kB 3.6 MB/s eta 0:00:00
     ------------------------------------ 157.0/157.0 kB 624.3 kB/s eta 0:00:00
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import openpyxl
from requests.exceptions import RequestException

# Load the spreadsheet
file_path = 'florida_companies.xlsx'
workbook = openpyxl.load_workbook(file_path)
sheet = workbook.active

# Define the column index for the email addresses
email_column = sheet.max_column + 1

# Regex pattern to match email addresses
email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'

# Maximum number of retries for failed requests
max_retries = 3

# Iterate over each row in the spreadsheet
for row_index, row in enumerate(sheet.iter_rows(min_row=2, values_only=True), start=2):
    url = row[2]  # Assuming the URLs are in the first column

    retries = 0
    success = False

    while retries < max_retries and not success:
        try:
            # Send a GET request to the website
            response = requests.get(url)
            success = True

        except RequestException as e:
            print(f"Request failed for URL: {url}")
            print(f"Error: {e}")
            retries += 1

    if success:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find email addresses using regex
        email_addresses = re.findall(email_pattern, soup.get_text())

        # Join the found email addresses into a single string
        email_string = ', '.join(email_addresses)

        # Add the email addresses to the spreadsheet
        sheet.cell(row=row_index, column=email_column).value = email_string
    else:
        print(f"Skipping URL: {url}")

# Save the modified spreadsheet
workbook.save(file_path)
